# アナログフィルタの設計（Analog Filter Design)
バタワース，チェビシェフフィルタの設計を示す。この際，次を共通して用いる。<br>
scipy.signal.iirfilter https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.iirfilter.html  

アナログフィルタの周波数応答は次を用いる。<br>
scipy.signal.freqs  https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.freqs.html<br>
この二つのページ共に，[rad/s]で計算することを述べている。

In [ ]:
# coding: utf-8
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
%matplotlib inline

FLAG_fig = False

### フィルタ仕様
ローパスフィルタの場合　カットオフ周波数 wc[rad/s]，周波数レンジ（範囲）は $[10^0 ,10^3]$ とする<br>
バンドパスフィルタの場合は，スクリプト参照

In [ ]:
wc = 100 #Cut off  [rad/s]
W_range = np.logspace(0, 3, 100) # [10^0 ,10^3] [rad/s], 100 samples

### バタワースフィルタ（Butterworth filter）
バタワースフィルタはIIRフィルタの一種であり，有理多項式となるため，分子と分母の係数を求める。

#### ローパスフィルタ

In [ ]:
fig = plt.subplots(figsize=(8,4))

for order in [1, 2, 3, 4, 5]:
    b, a = signal.iirfilter(N = order, Wn = wc, 
                            btype='lowpass', analog=True, ftype='butter')
#    print('b = ', b) # numerator coefficient
#    print('a = ', a) # denominator coefficient
    w, h = signal.freqs(b, a, W_range)
    gain = 20*np.log10(abs(h))
    plt.plot(w, gain, label=('order '+str(order)) )

plt.xscale('log')
plt.ylim(-30, 0)
plt.grid(which = "both", axis='both', ls="--")
plt.xlabel('omega [rad / s]')
plt.ylabel('Gain [dB]')

plt.axvline(wc, color='g', ls='--') # cutoff frequency
plt.axhline(-3, color='g', label='-3 [dB]', ls='--') # -3 dB

plt.margins(0, 0.1) #これが無いと，横軸左端の目盛り 10^0 の左横に少し空白が生じる
plt.legend()

if FLAG_fig: plt.savefig('fig_AFLT_Butter_Lowpass_1.png')
plt.show()

上のグラフの横軸の単位 [rad/s]が見づらい場合には，$2 \pi f = \omega$ （$f$ [Hz],  $\omega$ [rad/s]）の関係を用いて，スクリプト中で  frq = w/(2*np.pi)  とおく。  frqの単位は[Hz]であり， plt.plot(frq, gain) とすれば，配列の要素順は変わらないので，横軸が[Hz]のグラフとなる

In [ ]:
fig = plt.subplots(figsize=(8,4))

f_range = np.logspace(0, 3, 100)

for order in [1, 2, 3, 4, 5]:
    b, a = signal.iirfilter(N = order, Wn = wc, 
                            btype='lowpass', analog=True, ftype='butter')
    w, h = signal.freqs(b, a, W_range)
    frq = w/(2*np.pi)
    gain = 20*np.log10(abs(h))
    plt.plot(frq, gain, label=('order '+str(order)) )

plt.xscale('log')
plt.ylim(-30, 0)
plt.grid(which = "both", axis='both', ls="--")
plt.xlabel('f [Hz]')
plt.ylabel('Gain [dB]')
plt.axhline(-3, color='g', label='-3 [dB]', ls='--') # -3 dB

plt.margins(0, 0.1) #これが無いと，横軸左端の目盛り 10^0 の左横に少し空白が生じる
plt.legend()

if FLAG_fig: plt.savefig('fig_AFLT_Butter_Lowpass_2.png')
plt.show()

#### バンドバスフィルタ
Ref. http://scipy-cookbook.readthedocs.io/items/ButterworthBandpass.html

In [ ]:
wc_L = 500.0    # 下限カットオフ周波数　[rad/s]
wc_H = 1500.0   # 上限カットオフ周波数  [rad/s]
W_linear_range = np.arange(0,2500) # linear scale

for order in [3, 6, 9]:
    b, a = signal.iirfilter(N=order, Wn=[wc_L, wc_H], btype='bandpass', analog=True, ftype='butter')
    w, h = signal.freqs(b, a, W_linear_range)
    plt.plot(w, abs(h), label=('order '+str(order)) )

plt.ylim(0, 1.2)
plt.xlim(0, 2500)
plt.xlabel('omega [rad / s]')
plt.ylabel('Gain [Linear]')

plt.axhline(1/np.sqrt(2), color='g', label='1/sqrt(2) gain', ls='--') # 1/sqrt(2) -> -3 dB
plt.legend()
plt.grid()

if FLAG_fig: plt.savefig('fig_AFLT_Butter_Bandpass_1.png')
plt.show()

#### ハイパスフィルタは各自で考えてください

### チェビシェフ多項式（Chebyshev polynomial）
https://en.wikipedia.org/wiki/Chebyshev_polynomials <br>
https://docs.scipy.org/doc/numpy/reference/routines.polynomials.chebyshev.html  
この多項式が，次のチェビシェフフィルタに用いられる。

In [ ]:
x = np.linspace(-np.pi/2, np.pi/2, num=100)

In [ ]:
c0 = np.polynomial.chebyshev.chebval(x,[1])
c2 = np.polynomial.chebyshev.chebval(x,[0,0,1])
c4 = np.polynomial.chebyshev.chebval(x,[0,0,0,0,1])
c6 = np.polynomial.chebyshev.chebval(x,[0,0,0,0,0,0,1])

In [ ]:
fig = plt.subplots(figsize=(12,4))
plt.plot(x, c0, label='order 0')
plt.plot(x, c2, label='order 2')
plt.plot(x, c4, label='order 4')
plt.plot(x, c6, label='order 6')

plt.title('Even order')
plt.xlim(x.min(), x.max())
plt.ylim(-3,3)
plt.grid()
plt.legend()

if FLAG_fig: plt.savefig('fig_Chebyshev_Polynomial_1.png')
plt.show()

In [ ]:
c1 = np.polynomial.chebyshev.chebval(x,[0,1])
c3 = np.polynomial.chebyshev.chebval(x,[0,0,0,1])
c5 = np.polynomial.chebyshev.chebval(x,[0,0,0,0,0,1])
c7 = np.polynomial.chebyshev.chebval(x,[0,0,0,0,0,0,0,1])

In [ ]:
fig = plt.subplots(figsize=(12,4))
#plt.plot(x,c0, x,c1, x,c3, x,c5)

plt.plot(x, c1, label='order 1')
plt.plot(x, c3, label='order 3')
plt.plot(x, c5, label='order 5')
plt.plot(x, c7, label='order 7')

plt.title('Odd order')
plt.xlim(x.min(), x.max())
plt.ylim(-3,3)
plt.grid()
plt.legend()

if FLAG_fig: plt.savefig('fig_Chebyshev_Polynomial_2.png')
plt.show()

### チェビシェフフィルタ（Chebyshev filter）
https://en.wikipedia.org/wiki/Chebyshev_filter  

第1種 <br>
$$ \textrm{ripple [dB]} = 20 \log_{10}\frac{1}{\sqrt{1+\varepsilon^2}} $$
この左辺を与えて，内部では$\epsilon$を計算して，これをチェビシェフ第1種フィルタの式に与えている。<br>
この式は次を参照：https://en.wikipedia.org/wiki/Chebyshev_filter　  
https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.cheby1.html

In [ ]:
fig = plt.subplots(figsize=(8,4))

ripple = 5 # [dB]
b, a = signal.iirfilter(N=4, Wn=wc, rp=ripple, 
                        btype='low', analog=True, ftype='cheby1')
w, h = signal.freqs(b, a, W_range)
plt.semilogx(w, 20 * np.log10(abs(h)))

plt.title('Chebyshev Type I frequency response (rp='+str(ripple)+')')
plt.xlabel('omega [rad / s]')
plt.ylabel('Gain [dB]')

plt.ylim(-30,1)
plt.grid(which='both', axis='both')

plt.axvline(wc, color='g', ls='--') # cutoff frequency
plt.axhline(-3, color='g', label='-3 [dB]', ls='--') # -3 dB
plt.axhline(-ripple, color='orangered', label='ripple', ls='--') # ripple

plt.legend()
plt.margins(0, 0.1)

if FLAG_fig: plt.savefig('fig_AFLT_Cheby1_Lowpass.png')
plt.show()

第2種　https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.cheby2.html

In [ ]:
fig = plt.subplots(figsize=(8,4))

ripple = 20 # [dB]
b, a = signal.iirfilter(N=4, Wn=wc, rs=ripple, btype='low', analog=True, ftype='cheby2')
w, h = signal.freqs(b, a, W_range)
plt.semilogx(w, 20 * np.log10(abs(h)))

plt.title('Chebyshev Type II frequency response (rs='+str(ripple)+')')
plt.xlabel('omega [rad / s]')
plt.ylabel('Gain [dB]')

plt.ylim(-30,1)
plt.grid(which='both', axis='both')

plt.axvline(wc, color='g', ls='--') # cutoff frequency
plt.axhline(-3, color='g', label='-3 [dB]', ls='--') # -3 dB
plt.axhline(-ripple, color='orangered', label='ripple', ls='--') # ripple

plt.margins(0, 0.1)

if FLAG_fig: plt.savefig('fig_AFLT_Cheby2_Lowpass.png')
plt.savefig('fig_AFLT_Cheby2_Lowpass.png')
plt.show()